In [1]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec
import cmasher as cmr

import numpy as np
import os, glob, atexit, signal
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=0
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle, ax_change_color
from importlib import reload
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value
from common_func import *

In [2]:
mode1 = 'nh'
database1 = f"/home/jeon/MissingSat/database/{mode1}"
iout1 = 1026
repo1, rurmode1, dp1 = mode2repo(mode1)
snap1 = uri.RamsesSnapshot(repo1, iout1, mode=rurmode1)
snap1s = uri.TimeSeries(snap1)
snap1s.read_iout_avail()
nout1 = snap1s.iout_avail['iout']; nout=nout1[nout1 <= iout1]
gals1 = uhmi.HaloMaker.load(snap1, galaxy=True, double_precision=dp1)
hals1 = uhmi.HaloMaker.load(snap1, galaxy=False, double_precision=dp1)

LG1 = pklload(f"{database1}/LocalGroup.pickle")
allsats1 = None; allsubs1 = None; states1 = None
keys1 = list(LG1.keys())
for key in keys1:
    sats = LG1[key]['sats']; subs = LG1[key]['subs']; real = LG1[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True; subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'; state[~ind] = 'pair'
    
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsats1 = sats if allsats1 is None else np.hstack((allsats1, sats))
    allsubs1 = subs if allsubs1 is None else np.hstack((allsubs1, subs))
    states1 = state if states1 is None else np.hstack((states1, state))
argsort = np.argsort(allsubs1['id'])
allsubs1 = allsubs1[argsort]; states1 = states1[argsort]
dinks1 = allsubs1[states1 == 'dink']
pairs1 = allsubs1[states1 == 'pair']
upairs1 = allsubs1[states1 == 'upair']

print(len(allsubs1), np.unique(states1, return_counts=True))  

1336 (array(['dink', 'pair', 'upair'], dtype='<U7'), array([1145,  130,   61]))


In [3]:
mode2 = 'nh2'
database2 = f"/home/jeon/MissingSat/database/{mode2}"
iout2 = 797
repo2, rurmode2, dp2 = mode2repo(mode2)
snap2 = uri.RamsesSnapshot(repo2, iout2, mode=rurmode2)
snap2s = uri.TimeSeries(snap2)
snap2s.read_iout_avail()
nout2 = snap2s.iout_avail['iout']; nout=nout2[nout2 <= iout2]
gals2 = uhmi.HaloMaker.load(snap2, galaxy=True, double_precision=dp2)
hals2 = uhmi.HaloMaker.load(snap2, galaxy=False, double_precision=dp2)

LG2 = pklload(f"{database2}/LocalGroup.pickle")
allsats2 = None; allsubs2 = None; states2 = None
keys2 = list(LG2.keys())
for key in keys2:
    sats = LG2[key]['sats']; subs = LG2[key]['subs']; real = LG2[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True; subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'; state[~ind] = 'pair'
    
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsats2 = sats if allsats2 is None else np.hstack((allsats2, sats))
    allsubs2 = subs if allsubs2 is None else np.hstack((allsubs2, subs))
    states2 = state if states2 is None else np.hstack((states2, state))
argsort = np.argsort(allsubs2['id'])
allsubs2 = allsubs2[argsort]; states2 = states2[argsort]
dinks2 = allsubs2[states2 == 'dink']
pairs2 = allsubs2[states2 == 'pair']
upairs2 = allsubs2[states2 == 'upair']

print(len(allsubs2), np.unique(states2, return_counts=True))

1242 (array(['dink', 'pair', 'upair'], dtype='<U7'), array([1120,   79,   43]))


In [4]:
stree1 = pklload(f"{database1}/stable_progenitors.pickle")
stree2 = pklload(f"{database2}/stable_progenitors.pickle")
rtree1 = pklload(f"{database1}/reduced_tree.pickle")
rtree2 = pklload(f"{database2}/reduced_tree.pickle")

In [5]:
stree1.dtype

dtype([('lastid', '<i2'), ('timestep', '<i2'), ('id', '<i2'), ('give_score', '<f8'), ('take_score', '<f8')])

In [13]:
stree2

array([( 644, 797,  644, 1., 1.), (1529, 797, 1529, 1., 1.),
       (1577, 797, 1577, 1., 1.), ..., (   0,   0,    0, 0., 0.),
       (   0,   0,    0, 0., 0.), (   0,   0,    0, 0., 0.)],
      dtype=[('lastid', '<i2'), ('timestep', '<i2'), ('id', '<i2'), ('give_score', '<f8'), ('take_score', '<f8')])

In [16]:
stree2[stree2['lastid']==644]['id']

array([  644, 20997,   638,   636,   631,   649,   640,   655,   659,
         654,   650,   647,   650,   634,   648,   642,   641,   637,
         651,   639,   652,   650,   656,   659,   657,   657,   661,
         649,   654,   650,   652,   652,   662,   667,   661,   665,
         665,   653,   655,   655,   656,   651,   649,   652,   654,
         649,   647,   653,   652,   655,   652,   654,   661,   661,
         653,   658,   665,   660,   649,   649,   664,   663,   647,
         650,   643,   640,   635,   646,   647,   646,   657,   654,
         672,   669,   667,   651,   645,   652,   642,   653,   655,
         638,   657,   662,   662,   664,   659,   646,   652,   660,
         662,   654,   664,   651,   644,   647,   648,   636,   648,
         643,   652,   650,   644,   664,   656,   645,   645,   644,
         652,   655,   642,   669,   677,   678,   662,   655,   653,
         663,   658,   663,   673,   681,   665,   669,   665,   671,
         647,   665,

In [17]:
rtree2[644]['id']

array([  644, 20997,   638,   636,   631,   649,   640,   655,   659,
         654,   650,   647,   650,   634,   648,   642,   641,   637,
         651,   639,   652,   650,   656,   659,   657,   657,   661,
         649,   654,   650,   652,   652,   662,   667,   661,   665,
         665,   653,   655,   655,   656,   651,   649,   652,   654,
         649,   647,   653,   652,   655,   652,   654,   661,   661,
         653,   658,   665,   660,   649,   649,   664,   663,   647,
         650,   643,   640,   635,   646,   647,   646,   657,   654,
         672,   669,   667,   651,   645,   652,   642,   653,   655,
         638,   657,   662,   662,   664,   659,   646,   652,   660,
         662,   654,   664,   651,   644,   647,   648,   636,   648,
         643,   652,   650,   644,   664,   656,   645,   645,   644,
         652,   655,   642,   669,   677,   678,   662,   655,   653,
         663,   658,   663,   673,   681,   665,   669,   665,   671,
         647,   665,

In [8]:
stree1[stree1['lastid']==693]

array([(693, 1026,   693, 1.        , 1.        ),
       (693,  982,   687, 1.        , 0.5492228 ),
       (693,  981,   690, 1.        , 0.54639175),
       (693,  980,   693, 1.        , 0.54358974),
       (693,  979,   699, 1.        , 0.52736318),
       (693,  978,   704, 1.        , 0.52736318),
       (693,  977,   685, 1.        , 0.53      ),
       (693,  976,   681, 1.        , 0.52736318),
       (693,  975,   693, 1.        , 0.52736318),
       (693,  974,   684, 1.        , 0.50717703),
       (693,  973,   687, 1.        , 0.50717703),
       (693,  972,   685, 1.        , 0.50717703),
       (693,  971,   684, 1.        , 0.50236967),
       (693,  970,   680, 1.        , 0.52475248),
       (693,  969,   687, 1.        , 0.51707317),
       (693,  968,   701, 1.        , 0.53      ),
       (693,  967,   700, 1.        , 0.51960784),
       (693,  966,   701, 1.        , 0.49765258),
       (693,  965,   696, 1.        , 0.50236967),
       (693,  964,   701, 1.   